<a href="https://colab.research.google.com/github/psb-david-petty/google-colaboratory/blob/master/partitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partitions

Based on a [Mathologer](http://youtu.be/iJ8pnCO0nTY) video, this Python script computes the number of unique partitions of a number using *Euler's pentagonal formula*.

- First compute the pentagonal numbers (using the [pentagonal number theorem](https://en.wikipedia.org/wiki/Pentagonal_number_theorem#Relation_with_partitions)). The [OEIS](https://oeis.org/) for pentagonal numbers is: [https://oeis.org/A001318](https://oeis.org/A001318).
- Use those in conjuction with a [Fibonacci](https://en.wikipedia.org/wiki/Fibonacci_number)-like recurrence rule to calculate the numbers of unique partitions. The [OEIS](https://oeis.org/) for partitions is: [https://oeis.org/A000041](https://oeis.org/A000041).

[*The Pentagonal Number Theorem and All That*](https://pages.uoregon.edu/koch/PentagonalNumbers.pdf) is a paper by [Dick Koch](https://pages.uoregon.edu/koch/) describing the theory and listing the partitions up to `part(200)`.

## Code

The code has some interesting features.

- `p` is a two-liner that generates the n<sup>th</sup> pentagonal number based on the formula `m * (3 * m - 1) // 2` where `m` is 0, 1, -1, 2, -2, 3, -3, ... for the various values of `n` &ge; 0.
- `pent` *should be* `return [ p(i) for i in range(n) if p(i) <= n ]`, but I wanted to get fancy and calculate the maximum `n` needed to include all `p(i) <= n`. I attempted to invert `p(i)` $\le$ `n`. Sadly, because the signs in `p(i)` alternate, this formula for `n` is not reliable, so `pent` is not used.

$$
\begin{align}
\frac{1}{2} \left[ 3 \left(\frac{n + 1}{2} \right)^{2} - \left(\frac{n + 1}{2} \right) \right] &\le x \\
\frac{3 \left( n + 1 \right)^{2}}{8} - \frac{\left( n + 1 \right)}{4} &\le x \\
3 \left( n + 1 \right)^{2} - 2 \left( n + 1 \right) &\le 8x \\
\left( 3 n + 3 \right) \left( n + 1 \right) - \left( 2 n + 2 \right) &\le 8 x \\
3 n^{2} + 4 n + 3 - 2 n - 2 &\le 8 x \\
3 n^{2} + 2 n + 1 &\le 8x \\
3 \left( n + \frac{1}{3} \right)^{2} + 1 - \frac{1}{3} &\le 8 x \\
3 \left( n + \frac{1}{3} \right)^{2} + \frac{2}{3} &\le 8 x \\
\left( n + \frac{1}{3} \right)^{2} + \frac{2}{9} &\le \frac{8}{3} x \\
\left( n + \frac{1}{3} \right)^{2} &\le \frac{8}{3} x - \frac{2}{9} \\
\left( n + \frac{1}{3} \right) &\le \sqrt{\frac{8}{3} x - \frac{2}{9}} \\
n&\le \sqrt{\frac{8}{3} x - \frac{2}{9}} - \frac{1}{3}\\
\end{align}
$$

- `part` uses a list of `p(i)` $\le$ `n` to recursively look *backwards* from the end of the current list to generate the next element of the list, generating the sum based on indices in the list of `p(i)` $\le$ `n`.


In [35]:
#!/usr/bin/env python3
#
# partitions.py
#
# http://youtu.be/iJ8pnCO0nTY
# https://pages.uoregon.edu/koch/PentagonalNumbers.pdf

# https://oeis.org/A001318
def p(n):
    """Return nth generalized pentagonal number."""
    m = (n + 1) // 2 * (-1) ** ((n + 1) % 2)
    return m * (3 * m - 1) // 2

def pent(n):
    """Return list of generalized pentagonal numbers <= n."""
    # This magic comes from inverting p(n) and creating a list w/ p(i) <= n (at least).
    # TODO: This function is not used in part(n) because x is off a bit.
    x = int(max(0, 8 / 3 * n - 2 / 9) ** (1 / 2) - 1 / 3) + 1
    if len([ p(i) for i in range(x) ]) < len([ p(i) for i in range(n) if p(i) <= n ]):
        print(f"len({[ p(i) for i in range(x) ]}) "
              f"< len({[ p(i) for i in range(n) if p(i) <= n ]})")
    return [ p(i) for i in range(x) ]

print('# pent')
for n in range(100):
    print(f"pent({n}) = {pent(n)}")

# https://oeis.org/A000041
def part(n):
    """Return list of n unique partitions from 1 ... n. part(n)[-1] is number
    of unique partitions of n (n > 0)."""
    pents = [ p(i) for i in range(n + 1) if p(i) < n + 1 ][1: ]
    parts = [1, 1, ]
    for i in range(n):
        parts.append(sum([ parts[-k] * (-1) ** (j // 2 % 2)
            for j, k in enumerate(pents) if k < len(parts) ]))
            # for j, k in enumerate(m for m in pents if m < len(parts)) ]))
    return parts[2: ]

print('# part')
for i in range(200):
    print(f"{i} {len(part(i))} {part(i)} {part(i)[-1] if part(i) else 0}")

for n in [100, 200, 666, 1000, 10000, ]:
    print(f"part({n}) = {part(n)[-1]}")


# pent
pent(0) = [0]
pent(1) = [0, 1]
pent(2) = [0, 1]
pent(3) = [0, 1, 2]
pent(4) = [0, 1, 2]
pent(5) = [0, 1, 2, 5]
pent(6) = [0, 1, 2, 5]
len([0, 1, 2, 5]) < len([0, 1, 2, 5, 7])
pent(7) = [0, 1, 2, 5]
pent(8) = [0, 1, 2, 5, 7]
pent(9) = [0, 1, 2, 5, 7]
pent(10) = [0, 1, 2, 5, 7]
pent(11) = [0, 1, 2, 5, 7, 12]
pent(12) = [0, 1, 2, 5, 7, 12]
pent(13) = [0, 1, 2, 5, 7, 12]
pent(14) = [0, 1, 2, 5, 7, 12]
len([0, 1, 2, 5, 7, 12]) < len([0, 1, 2, 5, 7, 12, 15])
pent(15) = [0, 1, 2, 5, 7, 12]
pent(16) = [0, 1, 2, 5, 7, 12, 15]
pent(17) = [0, 1, 2, 5, 7, 12, 15]
pent(18) = [0, 1, 2, 5, 7, 12, 15]
pent(19) = [0, 1, 2, 5, 7, 12, 15]
pent(20) = [0, 1, 2, 5, 7, 12, 15]
pent(21) = [0, 1, 2, 5, 7, 12, 15, 22]
pent(22) = [0, 1, 2, 5, 7, 12, 15, 22]
pent(23) = [0, 1, 2, 5, 7, 12, 15, 22]
pent(24) = [0, 1, 2, 5, 7, 12, 15, 22]
pent(25) = [0, 1, 2, 5, 7, 12, 15, 22]
len([0, 1, 2, 5, 7, 12, 15, 22]) < len([0, 1, 2, 5, 7, 12, 15, 22, 26])
pent(26) = [0, 1, 2, 5, 7, 12, 15, 22]
pent(27) = [0, 1, 2, 5, 

## The hardest 'what comes next?'

On the same [Mathologer](http://youtu.be/iJ8pnCO0nTY) video (@ 40:45), [Burkard Polster](https://en.wikipedia.org/wiki/Burkard_Polster) poses another 'what comes next?' challenge. The pattern is this:

$$
\begin{align}
+\;1 &= 1 \\
\boxed{1} \\
2 &= 2 \\
+\;1 &= 1 \times 1 \\
\boxed{3} \\
3 &= 3 \\
2 &= 2 \times 1 \\
2 &= 1 \times 2 \\
+\;1 &= 1 \times 1 \times 1 \\
\boxed{8} \\
4 &= 4 \\
3 &= 3 \times 1 \\
3 &= 1 \times 3 \\
4 &= 2 \times 2 \\
2 &= 2 \times 1 \times 1 \\
2 &= 1 \times 2 \times 1 \\
2 &= 1 \times 1 \times 2 \\
+\;1 &= 1 \times 1 \times 1 \times 1 \\
\boxed{21}
\end{align}
$$

So, what follows: $1, 3, 8, 21$ ?

## Code

https://oeis.org/A001906

In [10]:
#!/usr/bin/env python3
#
# product.py
#
import functools, itertools, operator

prod = lambda iter: functools.reduce(operator.mul, iter, 1)

def divide(n, dividers):
    """Return partition of n using dividers."""
    assert n == dividers[-1], f"mismatch: {n} {dividers}"
    ones = [1,] * n
    return [ ones[dividers[i]: dividers[i + 1]]
        for i in range(len(dividers) - 1) ]

# Collect partition pattern.
result = list()
for i in range(16):
    acc, r = 0, range(1, i)
    for j in range(0, i):
        parts = [ [0,] + list(x) + [i] for x in itertools.combinations(r, j) ]
        for part in parts:
            # print(f"{i} {j} {part} {divide(i, part)}")
            acc += prod([ sum(addend) for addend in divide(i, part) ])
    result.append(acc)
print(result)

close = lambda a, b: abs(a - b) <= max(1e-09 * max(abs(a), abs(b)), 0.0)
is_square = lambda x: int(x ** (1 / 2)) ** 2 == x

# Formula from https://oeis.org/A001906.
print([ n for n in range(1000000) if is_square(5 * n * n + 4)])


[0, 1, 3, 8, 21, 55, 144, 377, 987, 2584, 6765, 17711, 46368, 121393, 317811, 832040]
[0, 1, 3, 8, 21, 55, 144, 377, 987, 2584, 6765, 17711, 46368, 121393, 317811, 832040]
